In [1]:
#import relevant libraries
import os
import subprocess
import numpy as np
import pandas as pd
import sys; sys.path
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import explained_variance_score, r2_score
from sklearn.linear_model import Ridge

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, GroupKFold, GroupShuffleSplit

In [2]:
# read and clean HCP data
# set base dirctories
HCP_base_dir   = '/Users/elvishadhamala/Documents/yale/HCP/'

# load dataframes for the cortical areal-level properties
# suraface area, gray matter volume, and cortical thickness 
HCP_surf_area_df = pd.read_csv(os.path.join(HCP_base_dir, 'group_net_surfarea_native_freesurfer.csv'))
HCP_gray_vol_df = pd.read_csv(os.path.join(HCP_base_dir, 'group_net_grayvol_native_freesurfer.csv'))
HCP_thick_df = pd.read_csv(os.path.join(HCP_base_dir, 'group_net_thickavg_native_freesurfer.csv'))

# load subj behavioural and family data
HCP_subj_data_df = pd.read_csv(os.path.join(HCP_base_dir, 'hcp_behaviour.csv'))
HCP_subj_fam_df = pd.read_csv(os.path.join(HCP_base_dir, 'hcp_restricted.csv'))
HCP_subj_fs_df = pd.read_csv(os.path.join(HCP_base_dir, 'hcp_freesurfer.csv'))

#create a mask for the subject and family data and only keep the ones that have the cortical areal-level properties 
HCP_mask_data=np.isin(HCP_subj_data_df.Subject, HCP_surf_area_df.id)
HCP_subj_data_df = HCP_subj_data_df[HCP_mask_data]

HCP_mask_fam=np.isin(HCP_subj_fam_df.Subject, HCP_surf_area_df.id)
HCP_subj_fam_df = HCP_subj_fam_df[HCP_mask_fam]

HCP_mask_fs=np.isin(HCP_subj_fs_df.Subject, HCP_surf_area_df.id)
HCP_subj_fs_df = HCP_subj_fs_df[HCP_mask_fs]


HCP_surf_area_df = HCP_surf_area_df.set_index(HCP_subj_data_df.index)
HCP_gray_vol_df = HCP_gray_vol_df.set_index(HCP_subj_data_df.index)
HCP_thick_df = HCP_thick_df.set_index(HCP_subj_data_df.index)
HCP_subj_fs_df = HCP_subj_fs_df.set_index(HCP_subj_data_df.index)


HCP_icv = pd.DataFrame(HCP_subj_fs_df.FS_InterCranial_Vol)

# drop the id and 'none' columns
HCP_surf_area_df = HCP_surf_area_df.drop(columns=['id', 'lh_None', 'rh_None'])
HCP_gray_vol_df = HCP_gray_vol_df.drop(columns=['id', 'lh_None', 'rh_None'])
HCP_thick_df = HCP_thick_df.drop(columns=['id', 'lh_None', 'rh_None'])



HCP_cog = HCP_subj_data_df[["CogFluidComp_Unadj", "CogEarlyComp_Unadj", "CogTotalComp_Unadj", "CogCrystalComp_Unadj",
                  "PicSeq_Unadj", "CardSort_Unadj", "Flanker_Unadj", "PMAT24_A_CR", "ReadEng_Unadj", 
                  "PicVocab_Unadj", "ProcSpeed_Unadj", "DDisc_AUC_40K", "VSPLOT_TC", "SCPT_SEN", "SCPT_SPEC", 
                  "IWRD_TOT", "ListSort_Unadj", "MMSE_Score", "Language_Task_Math_Avg_Difficulty_Level", 
                  "Language_Task_Story_Avg_Difficulty_Level", "Relational_Task_Acc", "WM_Task_Acc"]] 


col_headers_main = ['Fluid Composite', 'Early Composite', 'Total Composite',
               'Crystal Composite', 'Visual Episodic Memory', 'Cognitive Flexibility (Card Sort)',
               'Inhibition (Flanker)', 'Fluid Intelligence (PMAT)', 'Reading Decoding', 
               'Vocabulary Comprehension', 'Processing Speed', 'Delay Discounting',
               'Spatial Orientation', 'Sustained Attention - Sens.', 
               'Sustained Attention - Spec.', 'Verbal Episodic Memory', 
               'Working Memory (List Sorting)', 'Cognitive Status', 'Arithmetic',
               'Story Comprehension', 'Relational Processing', 'Working Memory (N-Back)']

HCP_cog.columns = col_headers_main

HCP_cog = HCP_cog[['Fluid Composite', 'Total Composite', 'Crystal Composite',
                     'Visual Episodic Memory', 'Cognitive Flexibility (Card Sort)',
                     'Inhibition (Flanker)', 'Reading Decoding', 'Vocabulary Comprehension', 
                     'Processing Speed', 'Working Memory (List Sorting)']]



#get rid of all the subjects with nans
HCP_mask = np.asarray([~HCP_cog.isna().any(axis=1)])
HCP_surf_area = HCP_surf_area_df[np.transpose(HCP_mask==True)]
HCP_gray_vol = HCP_gray_vol_df[np.transpose(HCP_mask==True)]
HCP_thick = HCP_thick_df[np.transpose(HCP_mask==True)]
HCP_icv = HCP_icv[np.transpose(HCP_mask==True)]
HCP_cog = HCP_cog[np.transpose(HCP_mask==True)]
HCP_fam = HCP_subj_fam_df.loc[np.transpose(HCP_mask==True)]
HCP_subj = HCP_subj_data_df.loc[np.transpose(HCP_mask==True)]

# get normalised measured (by icv)
HCP_surf_area_norm = pd.DataFrame(HCP_surf_area.values/HCP_icv.values, columns=HCP_surf_area.columns)
HCP_gray_vol_norm = pd.DataFrame(HCP_gray_vol.values/HCP_icv.values, columns=HCP_gray_vol.columns)
HCP_thick_norm = pd.DataFrame(HCP_thick.values/HCP_icv.values, columns=HCP_thick.columns)


HCP_surfarea = HCP_surf_area.set_index(HCP_subj.index)
HCP_grayvol = HCP_gray_vol.set_index(HCP_subj.index)
HCP_thick = HCP_thick.set_index(HCP_subj.index)
HCP_surfarea_norm = HCP_surf_area_norm.set_index(HCP_subj.index)
HCP_grayvol_norm = HCP_gray_vol_norm.set_index(HCP_subj.index)
HCP_thick_norm = HCP_thick_norm.set_index(HCP_subj.index)
HCP_fam = HCP_fam.set_index(HCP_subj.index)

In [3]:
# read and clean up ABCD data
# set base dirctories
ABCD_base_dir   = '/Users/elvishadhamala/Documents/yale/ABCD'

# load dataframes for the cortical areal-level properties
# suraface area, gray matter volume, and cortical thickness 
ABCD_surf_area_df = pd.read_csv(os.path.join(ABCD_base_dir, 'ABCD_group_surfarea.csv'), header=None)
ABCD_gray_vol_df = pd.read_csv(os.path.join(ABCD_base_dir, 'ABCD_group_grayvol.csv'), header=None)
ABCD_thick_df = pd.read_csv(os.path.join(ABCD_base_dir, 'ABCD_group_thickavg.csv'), header=None)
ABCD_icv_df = pd.read_csv(os.path.join(ABCD_base_dir, 'ABCD_icv.csv'), header=None)

# load subj behavioural and family data
ABCD_subj = pd.read_csv(os.path.join(ABCD_base_dir, 'ABCD_1823_demo_cog.csv'))


ABCD_surfarea = ABCD_surf_area_df.T
ABCD_grayvol = ABCD_gray_vol_df.T
ABCD_thick = ABCD_thick_df.T


# get normalised measured (by icv)
ABCD_surfarea_norm = pd.DataFrame(ABCD_surfarea.values/ABCD_icv_df.values, columns=ABCD_surfarea.columns)
ABCD_grayvol_norm = pd.DataFrame(ABCD_grayvol.values/ABCD_icv_df.values, columns=ABCD_grayvol.columns)
ABCD_thick_norm = pd.DataFrame(ABCD_thick.values/ABCD_icv_df.values, columns=ABCD_thick.columns)


ABCD_cog = ABCD_subj
ABCD_cog = ABCD_cog.drop(columns=['subjectkey', 'src_subject_id', 'sex', 'race_ethnicity', 'site_id_l'])
ABCD_cog

col_headers_main = ['Vocabulary Comprehension', 'Inhibition (Flanker)', 'Working Memory (List Sorting)',
                   'Cognitive Flexibility (Card Sort)', 'Processing Speed', 'Visual Episodic Memory',
                   'Reading Decoding', 'Fluid Composite', 'Crystal Composite', 'Total Composite',
                   'RAVLT - Trial VI Correct', 'RAVLT - Trial VII Correct', 'WISC-V - Total Raw Score',
                   'LMT - % Correct', 'LMT - RT Correct', 'LMT Efficiency']

ABCD_cog.columns = col_headers_main

ABCD_cog = ABCD_cog[['Fluid Composite', 'Total Composite', 'Crystal Composite',
                     'Visual Episodic Memory', 'Cognitive Flexibility (Card Sort)',
                     'Inhibition (Flanker)', 'Reading Decoding', 'Vocabulary Comprehension', 
                     'Processing Speed', 'Working Memory (List Sorting)']]

,Fluid Composite,Total Composite,Crystal Composite,Visual Episodic Memory,Cognitive Flexibility (Card Sort),Inhibition (Flanker),Reading Decoding,Vocabulary Comprehension,Processing Speed,Working Memory (List Sorting)
0,90.0,86.0,87.0,103.0,93.0,84.0,96.0,81.0,94.0,94.0
1,77.0,75.0,81.0,116.0,84.0,86.0,86.0,79.0,63.0,74.0
2,104.0,92.0,83.0,109.0,95.0,100.0,91.0,78.0,101.0,113.0
3,88.0,93.0,101.0,97.0,97.0,91.0,97.0,105.0,71.0,105.0
4,84.0,83.0,89.0,88.0,96.0,92.0,94.0,86.0,82.0,90.0
...,...,...,...,...,...,...,...,...,...,...
1818,88.0,88.0,94.0,95.0,90.0,94.0,100.0,89.0,92.0,90.0
1819,87.0,83.0,86.0,100.0,85.0,85.0,89.0,86.0,88.0,101.0
1820,83.0,85.0,92.0,124.0,86.0,75.0,95.0,91.0,74.0,86.0
1821,101.0,93.0,89.0,111.0,98.0,102.0,88.0,92.0,99.0,97.0


In [ ]:
#number of repetitions you want to perform
rep = 10
#number of folds you want in the cross-validation
k = 3
#proportion of data you want in your training set and test set
train_size = .66
test_size = 1-train_size

#regression model type
regr = Ridge(normalize=True, max_iter=1000000, solver='lsqr')

#set hyperparameter grid space you want to search through for the model
#adapted from the Thomas Yeo Lab Github: 
#ThomasYeoLab/CBIG/blob/master/stable_projects/predict_phenotypes/He2019_KRDNN/KR_HCP/CBIG_KRDNN_KRR_HCP.m
alphas = [0, 0.00001, 0.0001, 0.001, 0.004, 0.007, 0.01, 0.04, 0.07, 0.1, 0.4, 0.7, 1, 1.5, 2, 2.5, 3,
          3.5, 4, 5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 100, 150, 200, 300, 500, 700, 1000, 10000]

#param grid set to the hyperparamters you want to search through
paramGrid ={'alpha': alphas}

#set x data to be the input variable you want to use
X_HCP = HCP_surfarea_norm
X_ABCD = ABCD_surfarea_norm

Y_HCP = HCP_cog
Y_ABCD = ABCD_cog

#todo: thick, thick_norm


#number of variables you want to predict to be the number of variables stored in the cognition variablse
n_cog = Y_HCP.shape[1]

#number of features 
n_feat = X_HCP.shape[1]

#number of test sets
n_test = 2

In [ ]:
#create arrays to store variables

#r^2 - coefficient of determination
r2_ABCD = np.zeros([rep,n_cog,n_test])
#explained variance
var_ABCD = np.zeros([rep,n_cog,n_test])
#correlation between true and predicted (aka prediction accuracy)
corr_ABCD = np.zeros([rep,n_cog,n_test])
#optimised alpha (hyperparameter)
opt_alpha_ABCD = np.zeros([rep,n_cog])
#predictions made by the model
#don't need to save any of these right now
#preds = np.zeros([rep,n_cog,int(np.ceil(X.shape[0]*test_size))])
#true test values for cognition
#cogtest = np.zeros([rep,n_cog,int(np.ceil(X.shape[0]*test_size))])
#feature importance extracted from the model
featimp_ABCD = np.zeros([rep,n_feat,n_cog])
#for when the feat weights get haufe-inverted
featimp_haufe_ABCD = np.zeros([rep,n_feat,n_cog])


#r^2 - coefficient of determination
r2_HCP = np.zeros([rep,n_cog,n_test])
#explained variance
var_HCP = np.zeros([rep,n_cog,n_test])
#correlation between true and predicted (aka prediction accuracy)
corr_HCP = np.zeros([rep,n_cog,n_test])
#optimised alpha (hyperparameter)
opt_alpha_HCP = np.zeros([rep,n_cog])
#predictions made by the model
#don't need to save any of these right now
#preds = np.zeros([rep,n_cog,int(np.ceil(X.shape[0]*test_size))])
#true test values for cognition
#cogtest = np.zeros([rep,n_cog,int(np.ceil(X.shape[0]*test_size))])
#feature importance extracted from the model
featimp_HCP = np.zeros([rep,n_feat,n_cog])
#for when the feat weights get haufe-inverted
featimp_haufe_HCP = np.zeros([rep,n_feat,n_cog])



In [ ]:
#iterate through number of models
for p in range(rep):
    #print model # you're on
    print('Model %d' %(p+1))
    #HCP
    #group split HCP male data into train and test sets, using family ID as group ccategory
    train_inds_HCP, test_inds_HCP = next(GroupShuffleSplit(test_size=1-train_size, n_splits=1, random_state = p).split(X_HCP, groups=HCP_fam['Family_ID']))
    #x_train, x_test, cog_train, cog_test = train_test_split(X, Y, test_size=1-train_size, shuffle=True, random_state=p)
    
    #set x values based on indices from split  
    x_train_HCP = X_HCP.iloc[train_inds_HCP].values
    x_test_HCP = X_HCP.iloc[test_inds_HCP].values
        
    #set y values based on indices from split      
    cog_train_HCP = Y_HCP.iloc[train_inds_HCP].values
    cog_test_HCP = Y_HCP.iloc[test_inds_HCP].values
    
    
    #ABCD
    train_inds_ABCD, test_inds_ABCD = next(GroupShuffleSplit(test_size=1-train_size, n_splits=1, random_state = p).split(X_ABCD, groups=ABCD_subj['site_id_l']))
    #x_train, x_test, cog_train, cog_test = train_test_split(X, Y, test_size=1-train_size, shuffle=True, random_state=p)
    
    #set x values based on indices from split
    x_train_ABCD = X_ABCD.iloc[train_inds_ABCD].values
    x_test_ABCD = X_ABCD.iloc[test_inds_ABCD].values
        
    #set y values based on indices from split  
    cog_train_ABCD = Y_ABCD.iloc[train_inds_ABCD].values
    cog_test_ABCD = Y_ABCD.iloc[test_inds_ABCD].values    
    
   
    
    #iterate through the cognitive metrics you want to predict
    for cog in range (n_cog):

        #print and set cognitive metrics being predicted 
        print ("Behaviour: %s" % Y_HCP.columns[cog])
    
        y_train_HCP = cog_train_HCP[:,cog]
        y_test_HCP = cog_test_HCP[:,cog]
        
        y_train_ABCD = cog_train_ABCD[:,cog]
        y_test_ABCD = cog_test_ABCD[:,cog]

        
        #store all the y_test values in a separate variable that can be accessed later if needed
        #cogtest[p,cog,:] = y_test

        ##HCP
        #create variables to store nested CV scores, and best parameters from hyperparameter optimisation
        best_scores_HCP = []
        best_params_HCP = []


        #set parameters for inner and outer loops for CV
        cv_split = GroupKFold(n_splits=k)
            
            
        #define regressor with grid-search CV for inner loop
        gridSearch_HCP = GridSearchCV(estimator=regr, param_grid=paramGrid, n_jobs=1, verbose=0, cv=cv_split, scoring='explained_variance')

        #fit regressor to the model, use family ID as group category again
        gridSearch_HCP.fit(x_train_HCP, y_train_HCP, groups=HCP_fam.iloc[train_inds_HCP]['Family_ID'])

        #save parameters corresponding to the best score
        best_params_HCP.append(list(gridSearch_HCP.best_params_.values()))
        best_scores_HCP.append(gridSearch_HCP.best_score_)
        
        
        ##ABCD
        #create variables to store nested CV scores, and best parameters from hyperparameter optimisation
        best_scores_ABCD = []
        best_params_ABCD = []        
        
        
        
        #set parameters for inner and outer loops for CV
        cv_split = GroupKFold(n_splits=k)
            
        #define regressor with grid-search CV for inner loop
        gridSearch_ABCD = GridSearchCV(estimator=regr, param_grid=paramGrid, n_jobs=1, verbose=0, cv=cv_split, scoring='explained_variance')

        #fit regressor to the model, use family ID as group category again
        gridSearch_ABCD.fit(x_train_ABCD, y_train_ABCD, groups=ABCD_subj.iloc[train_inds_ABCD]['site_id_l'])

        #save parameters corresponding to the best score
        best_params_ABCD.append(list(gridSearch_ABCD.best_params_.values()))
        best_scores_ABCD.append(gridSearch_ABCD.best_score_)
        
        
        
        #save optimised alpha values
        #opt_alpha[p,cog] = best_params[nested_scores.index(np.max(nested_scores))][0]
        #ends up just being a single value that's chosen by GridSearchCV here since it's no longer nested
        #but the line below just makes it easier to go back to a nested set-up if needed
        opt_alpha_HCP[p,cog] = best_params_HCP[best_scores_HCP.index(np.max(best_scores_HCP))][0]
        
        opt_alpha_ABCD[p,cog] = best_params_ABCD[best_scores_ABCD.index(np.max(best_scores_ABCD))][0]

        #fit model using optimised hyperparameter
        model_HCP = Ridge(alpha = opt_alpha_HCP[p,cog], normalize=True, max_iter=1000000, solver='lsqr')
        model_HCP.fit(x_train_HCP, y_train_HCP);
            
      
        model_ABCD = Ridge(alpha = opt_alpha_ABCD[p,cog], normalize=True, max_iter=1000000, solver='lsqr')
        model_ABCD.fit(x_train_ABCD, y_train_ABCD);
        
        
        #compute r^2 (coefficient of determination)
        r2_HCP[p,cog,0]=model_HCP.score(x_test_HCP,y_test_HCP)
        r2_HCP[p,cog,1]=model_HCP.score(x_test_ABCD,y_test_ABCD)
        
        
        r2_ABCD[p,cog,0]=model_ABCD.score(x_test_HCP,y_test_HCP)
        r2_ABCD[p,cog,1]=model_ABCD.score(x_test_ABCD,y_test_ABCD)
        
        
        preds_HCP = []
        preds_ABCD = []

        #generate predictions from HCP m model
        preds_HCP = model_HCP.predict(x_test_HCP).ravel()
        preds_ABCD = model_HCP.predict(x_test_ABCD).ravel()

        #compute explained variance 
        var_HCP[p,cog,0] = explained_variance_score(y_test_HCP, preds_HCP)
        var_HCP[p,cog,1] = explained_variance_score(y_test_ABCD, preds_ABCD)


        #compute correlation between true and predicted (prediction accuracy)
        corr_HCP[p,cog,0] = np.corrcoef(y_test_HCP.ravel(), preds_HCP)[1,0]
        corr_HCP[p,cog,1] = np.corrcoef(y_test_ABCD.ravel(), preds_ABCD)[1,0]

        

        preds_HCP = []
        preds_ABCD = []

        #generate predictions from ABCD m model
        preds_HCP = model_ABCD.predict(x_test_HCP).ravel()
        preds_ABCD = model_ABCD.predict(x_test_ABCD).ravel()
        
        #compute explained variance 
        var_ABCD[p,cog,0] = explained_variance_score(y_test_HCP, preds_HCP)
        var_ABCD[p,cog,1] = explained_variance_score(y_test_ABCD, preds_ABCD)


        #compute correlation between true and predicted (prediction accuracy)
        corr_ABCD[p,cog,0] = np.corrcoef(y_test_HCP.ravel(), preds_HCP)[1,0]
        corr_ABCD[p,cog,1] = np.corrcoef(y_test_ABCD.ravel(), preds_ABCD)[1,0]


        cov_x = []
        cov_y = []
        #extract feature importance
        featimp_HCP[p,:,cog] = model_HCP.coef_
        #compute Haufe-inverted feature weights
        cov_x = np.cov(np.transpose(x_train_HCP))
        cov_y = np.cov(y_train_HCP)
        featimp_haufe_HCP[p,:,cog] = np.matmul(cov_x,featimp_HCP[p,:,cog])*(1/cov_y)
    
        
        cov_x = []
        cov_y = []
        #extract feature importance
        featimp_ABCD[p,:,cog] = model_ABCD.coef_
        #compute Haufe-inverted feature weights
        cov_x = np.cov(np.transpose(x_train_ABCD))
        cov_y = np.cov(y_train_ABCD)
        featimp_haufe_ABCD[p,:,cog] = np.matmul(cov_x,featimp_ABCD[p,:,cog])*(1/cov_y)
        

In [ ]:
# save results
base_dir  = '/Users/elvishadhamala/Documents/yale/HCP_ABCD_preds_results'

np.save(os.path.join(base_dir, 'surfarea_norm_r2_HCP.npy'),r2_HCP)
np.save(os.path.join(base_dir, 'surfarea_norm_var_HCP.npy'),var_HCP)
np.save(os.path.join(base_dir, 'surfarea_norm_corr_HCP.npy'),corr_HCP)
np.save(os.path.join(base_dir, 'surfarea_norm_alpha_HCP.npy'),opt_alpha_HCP)
np.save(os.path.join(base_dir, 'surfarea_norm_featimp_HCP.npy'),featimp_HCP)
np.save(os.path.join(base_dir, 'surfarea_norm_featimp_haufe_HCP.npy'),featimp_haufe_HCP)

np.save(os.path.join(base_dir, 'surfarea_norm_r2_ABCD.npy'),r2_ABCD)
np.save(os.path.join(base_dir, 'surfarea_norm_var_ABCD.npy'),var_ABCD)
np.save(os.path.join(base_dir, 'surfarea_norm_corr_ABCD.npy'),corr_ABCD)
np.save(os.path.join(base_dir, 'surfarea_norm_alpha_ABCD.npy'),opt_alpha_ABCD)
np.save(os.path.join(base_dir, 'surfarea_norm_featimp_ABCD.npy'),featimp_ABCD)
np.save(os.path.join(base_dir, 'surfarea_norm_featimp_haufe_ABCD.npy'),featimp_haufe_ABCD)

